In [78]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import pandas as pd

%matplotlib inline


In [79]:
imagem = cv2.imread('Imagens\IMG (1).bmp',0)
#plt.imshow(imagem)


In [80]:
#Colocando bordas na imagem
imagem = cv2.copyMakeBorder(imagem, 50, 50, 50, 50, cv2.BORDER_REPLICATE)
imagem_original = imagem

#Borra bastante
imagem = cv2.GaussianBlur(imagem,(75,75),0)
#plt.imshow(imagem, cmap='gray')

Limiar binary invertido + limiar de OTSU
o método de Otsu determina um valor de limite global ideal a partir do histograma da imagem

In [81]:
limiar, imagem_limiar = cv2.threshold(imagem,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU )

imagem_limiar_255 = imagem_limiar*255
imagem_limiar_255 = imagem_limiar_255.astype(np.uint8)
#plt.imshow(imagem_limiar_255, cmap='gray')

In [82]:

# Encontrando os contornos na imagem
contornos, void = cv2.findContours(imagem_limiar, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Desenhar os contornos na imagem original
#imagem_contornos = cv2.drawContours(cv2.cvtColor(imagem_original, cv2.COLOR_BAYER_BG2BGRA), contornos, -1, (255,0,0), -5)

#lt.imshow(imagem_contornos)

Aqui temos o numero de objetos na imagem, que vai ser igual ao numero de bordas encontradas
    - vai dar ruim quando um objeto esta em cima de outro, ai não tem o que fazer

In [83]:
size = len(contornos)
size

8

In [84]:
df = pd.DataFrame()


for cnt in contornos:
    #area
    area = cv2.contourArea(cnt)
    
    #perimetro
    perimetro = cv2.arcLength(cnt,True)

    #compacidade
    compacidade = (perimetro**2)/area
    print("Compacidade: ", compacidade)

    #circulatidade
    circularidade = (4*np.pi*area)/perimetro**2
    print("circulatidade: ", circularidade)


    #solidez
    hull = cv2.convexHull(cnt)
    hull_area = cv2.contourArea(hull)
    solidity = float(area)/hull_area
    print("Solidez: ", solidity)

    #Cria eclipse envolvente 
    (xc,yc), (d1,d2), angle  = cv2.fitEllipse(cnt)

    #Pegas os pontos do eixo maior
    rmajor = max(d1,d2)/2
    if angle > 90:
        angle = angle - 90
    else:
        angle = angle + 90

    x1 = xc + math.cos(math.radians(angle))*rmajor
    y1 = yc + math.sin(math.radians(angle))*rmajor
    x2 = xc + math.cos(math.radians(angle+180))*rmajor
    y2 = yc + math.sin(math.radians(angle+180))*rmajor
    #calcula distancia
    eixo_maior = np.sqrt((x2-x1)**2 + (y2-y1)**2)

    #Pegas os pontos do eixo menor
    rminor = min(d1,d2)/2
    if angle > 90:
        angle = angle - 90
    else:
        angle = angle + 90

    x1 = xc + math.cos(math.radians(angle))*rminor
    y1 = yc + math.sin(math.radians(angle))*rminor
    x2 = xc + math.cos(math.radians(angle+180))*rminor
    y2 = yc + math.sin(math.radians(angle+180))*rminor
    #calcula distancia
    eixo_menor = np.sqrt((x2-x1)**2 + (y2-y1)**2)
   
    #Razão do eixo menor pelo eixo maior
    razao_eixos = eixo_menor/eixo_maior
    print("Razão Eixos: ", razao_eixos)

    #Proporção
    x,y,w,h = cv2.boundingRect(cnt)
    

    rect_area = w*h
    extent = float(area)/rect_area
    print("Extent : ", extent)

    
    mask = np.zeros(imagem.shape,np.uint8)
    mask = cv2.drawContours(mask,[cnt],0,255,-1)
    #pixelpoints = np.transpose(np.nonzero(mask))

    mean_val = cv2.mean(imagem,mask = mask)
    print(mean_val[0])
    

   
    df = df.append({'Compacidade': [compacidade], 'Circularidade': [circularidade]}, ignore_index=True)

    

    #print(df.shape)

    #df.to_excel('/home/pi/Área de Trabalho/saidaCamera3.xlsx', index = False)

Compacidade:  17.593852125534255
circulatidade:  0.7142478250184555
Solidez:  0.9783509742061607
Razão Eixos:  0.6290194223429073
Extent :  0.6602924900118777
73.22259821555596
Compacidade:  17.493575351274114
circulatidade:  0.718342040550557
Solidez:  0.9900445790658076
Razão Eixos:  0.5970973703011282
Extent :  0.6608081096516276
47.16151273652628
Compacidade:  24.62674146972411
circulatidade:  0.5102733802524444
Solidez:  0.9397854763445388
Razão Eixos:  0.3654295785298962
Extent :  0.5992791211478172
56.027620427381386
Compacidade:  17.68359114495889
circulatidade:  0.7106232275643571
Solidez:  0.9528082437669882
Razão Eixos:  0.7376462179619397
Extent :  0.6404120213520328
49.86570194897494
Compacidade:  28.935043144160094
circulatidade:  0.4342959003638264
Solidez:  0.9574367145261198
Razão Eixos:  0.2738380971574839
Extent :  0.4548203017366013
48.30200053024983
Compacidade:  17.711818306173214
circulatidade:  0.7094907138912628
Solidez:  0.9892891421970424
Razão Eixos:  0.5298

C:\Users\bianc\AppData\Local\Temp\ipykernel_13404\2335983434.py:79: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Compacidade': [compacidade], 'Circularidade': [circularidade]}, ignore_index=True)
C:\Users\bianc\AppData\Local\Temp\ipykernel_13404\2335983434.py:79: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Compacidade': [compacidade], 'Circularidade': [circularidade]}, ignore_index=True)
C:\Users\bianc\AppData\Local\Temp\ipykernel_13404\2335983434.py:79: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Compacidade': [compacidade], 'Circularidade': [circularidade]}, ignore_index=True)
C:\Users\bianc\AppData\Local\Temp\ipykernel_13404\2335983434.py:79: FutureWarning: The fram